<a href="https://colab.research.google.com/github/nalgo-intern/team-a-2024-summer-08-26/blob/model/yoshitomi_model_rent_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#実装

##ライブラリ

In [1]:
from google.colab import drive
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error,  r2_score

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

##機械学習モデルの作成

In [61]:
# Google Driveをマウント
drive.mount('/content/drive')

# ファイルパスを指定してCSVファイルを読み込む
file_path = '/content/drive/MyDrive/Colab Notebooks/0829_train.csv'
train= pd.read_csv(file_path)

# データの表示
print(train.head())
train.shape

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   23区  築年数  階数  階        賃料    管理費    間取り        敷金        礼金   専有面積   最寄駅  \
0  葛飾区   13   5  2  123000.0   7000   1LDK  123000.0  123000.0  46.47   綾瀬駅   
1  葛飾区   13   5  5  128000.0   7000   1LDK  128000.0  128000.0  47.52   綾瀬駅   
2  荒川区   20  10  3  170000.0  12000   3LDK  170000.0  170000.0  69.83  三ノ輪駅   
3  荒川区   18   4  1  144000.0   9000   3LDK  144000.0  288000.0  58.93   尾久駅   
4  荒川区   18   4  4  160000.0  10000  3SLDK  160000.0  320000.0  66.54   尾久駅   

   23区_label  最寄駅_label  賃料_管理費合計  間取り_label  部屋数  LDK  S  
0         20        344  130000.0          2    1    3  0  
1         20        344  135000.0          2    1    3  0  
2         19          6  182000.0         17    3    3  0  
3         19        159  153000.0         17    3    3  0  
4         19        159  170000.0         19    3    3  1  


(42872, 18)

In [41]:
train

,23区,築年数,階数,階,賃料,管理費,間取り,敷金,礼金,専有面積,間取り_label,部屋数,LDK,S,23区_label,賃料_管理費合計
0,葛飾区,13,5,2,123000.0,7000,1LDK,123000.0,123000.0,46.47,2,1,3,0,20,130000.0
1,葛飾区,13,5,5,128000.0,7000,1LDK,128000.0,128000.0,47.52,2,1,3,0,20,135000.0
2,荒川区,20,10,3,170000.0,12000,3LDK,170000.0,170000.0,69.83,17,3,3,0,19,182000.0
3,荒川区,18,4,1,144000.0,9000,3LDK,144000.0,288000.0,58.93,17,3,3,0,19,153000.0
4,荒川区,18,4,4,160000.0,10000,3SLDK,160000.0,320000.0,66.54,19,3,3,1,19,170000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42867,中央区,29,3,3,95000.0,5000,1LDK,95000.0,95000.0,45.10,2,1,3,0,1,100000.0
42868,中央区,29,3,2,110000.0,5000,2LDK,110000.0,110000.0,58.44,11,2,3,0,1,115000.0
42869,中央区,30,3,3,116000.0,4000,2LDK,116000.0,116000.0,64.53,11,2,3,0,1,120000.0
42870,中央区,0,4,2,198000.0,8000,2LDK,198000.0,396000.0,59.87,11,2,3,0,1,206000.0


In [62]:
features = ['築年数', '階数', '階', '間取り_label', '部屋数', 'LDK', 'S','23区_label','最寄駅_label']

In [63]:
target = '賃料_管理費合計'

X = train[features]
y = train[target]

In [64]:
print(np.shape(X))
print(np.shape(y))

(42872, 9)
(42872,)


In [65]:
#標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [66]:
# トレーニングデータとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

###線形回帰モデル

In [75]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 線形回帰モデル
lr = LinearRegression()
lr.fit(X_train, y_train)

lr_train_pred = lr.predict(X_train)
lr_valid_pred = lr.predict(X_test)

# 評価指標の計算
mae_train = mean_absolute_error(y_train, lr_train_pred)
r2_train = r2_score(y_train, lr_train_pred)


mae_valid = mean_absolute_error(y_test, lr_valid_pred)
r2_valid = r2_score(y_test, lr_valid_pred)

# ベースラインモデルの評価 (平均値モデル)
baseline_pred = [y_train.mean()] * len(y_test)
baseline_mae = mean_absolute_error(y_test, baseline_pred)


# 精度
percentage = 100 * (1 - mae_valid / baseline_mae)


# 結果の表示
print(f"Train MAE: {mae_train}")
print(f"Train R^2: {r2_train}")


print(f"Valid MAE: {mae_valid}")
print(f"Valid R^2: {r2_valid}")
print(f'Performance Percentage: {percentage:.2f}%')


Train MAE: 30504.42568197674
Train R^2: 0.6314644528798916
Valid MAE: 31191.243929609365
Valid R^2: 0.6082662192803046
Performance Percentage: 49.09%


###GBDT(xgboost)

In [76]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# XGBoostの回帰モデル
xgb = XGBRegressor(random_state=0)
xgb.fit(X_train, y_train)

# 予測
xgb_train_pred = xgb.predict(X_train)
xgb_test_pred = xgb.predict(X_test)

# 評価指標の計算
mae_train = mean_absolute_error(y_train, xgb_train_pred)
r2_train = r2_score(y_train, xgb_train_pred)


mae_test = mean_absolute_error(y_test, xgb_test_pred)
r2_test = r2_score(y_test, xgb_test_pred)


# ベースラインモデルの評価 (平均値モデル)
baseline_pred = [y_train.mean()] * len(y_test)
baseline_mae = mean_absolute_error(y_test, baseline_pred)


# 精度
percentage = 100 * (1 - mae_valid / baseline_mae)

# 結果の表示
print(f"Train MAE: {mae_train}")
print(f"Train R^2: {r2_train}")

print(f"Test MAE: {mae_test}")
print(f"Test R^2: {r2_test}")
print(f'Performance Percentage: {percentage:.2f}%')

Train MAE: 15496.66090066332
Train R^2: 0.9194670136992844
Test MAE: 18786.31274433044
Test R^2: 0.8565853688646576
Performance Percentage: 49.09%


In [77]:
"""線形回帰モデルとGBDTのアンサンブル"""
train_pred = (lr_train_pred + xgb_train_pred) / 2
valid_pred = (lr_valid_pred + xgb_test_pred) / 2

# 評価指標の計算
mae_train = mean_absolute_error(y_train, train_pred)
r2_train = r2_score(y_train, train_pred)

mae_valid = mean_absolute_error(y_test, valid_pred)
r2_valid = r2_score(y_test, valid_pred)

# ベースラインモデルの評価 (平均値モデル)
baseline_pred = [y_train.mean()] * len(y_test)
baseline_mae = mean_absolute_error(y_test, baseline_pred)


# 精度
percentage = 100 * (1 - mae_valid / baseline_mae)

# 結果の表示
print(f"Train MAE: {mae_train}")
print(f"Train R^2: {r2_train}")

print(f"Valid MAE: {mae_valid}")
print(f"Valid R^2: {r2_valid}")
print(f'Performance Percentage: {percentage:.2f}%')

Train MAE: 21336.01838607864
Train R^2: 0.8318607942073931
Valid MAE: 23055.780174865264
Valid R^2: 0.7887536026190588
Performance Percentage: 62.37%


###LightGBM

In [67]:
# LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# LightGBMのハイパーパラメータ
params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# モデルのトレーニング
model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[test_data])

# 予測
y_pred = model.predict(X_test, num_iteration=model.best_iteration)


# モデルの評価
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.092483 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 499
[LightGBM] [Info] Number of data points in the train set: 30010, number of used features: 9
[LightGBM] [Info] Start training from score 146535.364245
Mean Absolute Error: 17922.00547364523


In [69]:
# モデルの評価
model_mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# ベースラインモデルの評価 (平均値モデル)
baseline_pred = [y_train.mean()] * len(y_test)
baseline_mae = mean_absolute_error(y_test, baseline_pred)


# 精度
percentage = 100 * (1 - model_mae / baseline_mae)
print(f'Model MAE: {model_mae}')
print(f'Baseline MAE: {baseline_mae}')
print(f'Performance Percentage: {percentage:.2f}%')
print(f'R² (Coefficient of Determination): {r2:.4f}')

Model MAE: 17922.00547364523
Baseline MAE: 61265.51753781756
Performance Percentage: 70.75%
R² (Coefficient of Determination): 0.8671


#家賃推定テスト


In [74]:
new_data = pd.DataFrame({
    '築年数': [13],
    '階数': [5],
    '階': [2],
    '間取り': ['1LDK'],
    '23区':['葛飾区'],
    '最寄駅':['綾瀬駅']
})

le = LabelEncoder()
new_data['間取り_label'] = le.fit_transform(new_data['間取り'])
new_data['23区_label'] = le.fit_transform(new_data['23区'])
new_data['最寄駅_label'] = le.fit_transform(new_data['最寄駅'])


new_data['部屋数'] = new_data['間取り'].apply(lambda x: 1 if 'ワンルーム' in x else int(x[0]) if x[0].isdigit() else 0)
new_data['LDK'] = new_data['間取り'].apply(lambda x: sum([x.count(c) for c in 'LDK']))
new_data['S'] = new_data['間取り'].apply(lambda x: 1 if 'S' in x else 0)

# 標準化
X_new = new_data[features]
X_new_scaled = scaler.transform(X_new)

#賃料の推定
y_pred = model.predict(X_new_scaled, num_iteration=model.best_iteration)

print('推定賃料:', y_pred[0])

推定賃料: 148965.26187435992


#補足

In [40]:
"""['間取り']のデータ処理"""

# selected_data['間取り_label'] = LabelEncoder().fit_transform(selected_data['間取り'])

# # 部屋数、LDK数、Sの有無を特徴量として追加
# # ワンルームは部屋数を1に設定
# selected_data['部屋数'] = selected_data['間取り'].apply(lambda x: 1 if 'ワンルーム' in x else int(x[0]) if x[0].isdigit() else 0)
# selected_data['LDK'] = selected_data['間取り'].apply(lambda x: sum([x.count(c) for c in 'LDK']))
# selected_data['S'] = selected_data['間取り'].apply(lambda x: 1 if 'S' in x else 0)

"""['23区']のデータ処理"""
# le = LabelEncoder()
# selected_data['23区_label'] = le.fit_transform(selected_data['23区'])

"""['賃料']['管理費']のデータ処理"""
#selected_data['賃料_管理費合計'] = selected_data['賃料'] + selected_data['管理費']

"['間取り']のデータ処理"